 # Paso 1: Importar las librerías necesarias
Comenzamos importando las librerías que utilizaremos en el proceso.
pandas y numpy para manipulación de datos.
random para selecciones aleatorias.
train_test_split de sklearn para dividir los datos.
Sequential, Dense de Keras para construir el modelo.
Métricas y callbacks para evaluar y optimizar el modelo.


In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.metrics import Recall, Precision
from tensorflow.keras.callbacks import EarlyStopping


ModuleNotFoundError: No module named 'tensorflow'

# Paso 2: Cargar y preparar los datos
## 2.1 Cargar el archivo resultados.csv
Este archivo debe contener las columnas ID_Pelicula y ID_Lista.

In [ ]:
# Cargar el archivo resultados.csv
data = pd.read_csv('resultados.csv')


## 2.2 Crear un mapeo de películas y listas
Creamos:

Un conjunto de todos los IDs de películas.
Un diccionario que mapea cada ID_Lista a un conjunto de ID_Pelicula.

In [ ]:
# Obtener el conjunto de todos los IDs de películas
all_movie_ids = set(data['ID_Pelicula'].astype(str))

# Crear un diccionario que mapea ID_Lista a un conjunto de IDs de películas
list_to_movies = data.groupby('ID_Lista')['ID_Pelicula'].apply(set).to_dict()

## 2.3 Crear un índice para los IDs de películas
Asignamos un índice único a cada película para crear los vectores booleanos.

In [ ]:
# Crear un diccionario que mapea ID_Pelicula a un índice único
movie_id_to_index = {movie_id: idx for idx, movie_id in enumerate(all_movie_ids)}
index_to_movie_id = {idx: movie_id for movie_id, idx in movie_id_to_index.items()}

# Número total de películas
num_movies = len(movie_id_to_index)


# Paso 3: Dividir las listas en entrenamiento y prueba
Dividimos las listas (IDs de listas) en conjuntos de entrenamiento y prueba (50% cada uno).

In [ ]:
# Obtener todos los IDs de listas
all_list_ids = list(list_to_movies.keys())

# Dividir en entrenamiento y prueba
train_list_ids, test_list_ids = train_test_split(all_list_ids, test_size=0.5, random_state=42)


# Paso 4: Crear los conjuntos de datos X e Y para entrenamiento
4.1 Inicializar listas para X e Y

In [ ]:
X_train = []
Y_train = []


## 4.2 Iterar sobre las listas de entrenamiento
Para cada lista en el conjunto de entrenamiento:

Seleccionar aleatoriamente entre 2 y 5 películas de la lista para el vector de entrada x.
Crear el vector de salida y marcando todas las películas de la lista.
Agregar x y y a X_train y Y_train.

In [ ]:
for list_id in train_list_ids:
    movies_in_list = list(list_to_movies[list_id])
    num_movies_in_list = len(movies_in_list)
    
    if num_movies_in_list < 2:
        continue  # Saltar listas con menos de 2 películas
    
    # Seleccionar aleatoriamente entre 2 y 5 películas para x
    num_movies_in_x = min(random.randint(2, 5), num_movies_in_list)
    movies_in_x = random.sample(movies_in_list, num_movies_in_x)
    
    # Crear vector x
    x = np.zeros(num_movies)
    for movie_id in movies_in_x:
        idx = movie_id_to_index[str(movie_id)]
        x[idx] = 1
    
    # Crear vector y
    y = np.zeros(num_movies)
    for movie_id in movies_in_list:
        idx = movie_id_to_index[str(movie_id)]
        y[idx] = 1
    
    # Agregar a los conjuntos de datos
    X_train.append(x)
    Y_train.append(y)


# Paso 5: Crear los conjuntos de datos X e Y para prueba
Repetimos el mismo proceso para las listas de prueba.

In [ ]:
X_test = []
Y_test = []

for list_id in test_list_ids:
    movies_in_list = list(list_to_movies[list_id])
    num_movies_in_list = len(movies_in_list)
    
    if num_movies_in_list < 2:
        continue  # Saltar listas con menos de 2 películas
    
    # Seleccionar aleatoriamente entre 2 y 5 películas para x
    num_movies_in_x = min(random.randint(2, 5), num_movies_in_list)
    movies_in_x = random.sample(movies_in_list, num_movies_in_x)
    
    # Crear vector x
    x = np.zeros(num_movies)
    for movie_id in movies_in_x:
        idx = movie_id_to_index[str(movie_id)]
        x[idx] = 1
    
    # Crear vector y
    y = np.zeros(num_movies)
    for movie_id in movies_in_list:
        idx = movie_id_to_index[str(movie_id)]
        y[idx] = 1
    
    # Agregar a los conjuntos de datos
    X_test.append(x)
    Y_test.append(y)


# Paso 6: Convertir las listas a arrays de NumPy

In [ ]:
X_train = np.array(X_train)
Y_train = np.array(Y_train)

X_test = np.array(X_test)
Y_test = np.array(Y_test)


# Paso 7: Construir y entrenar el modelo con Keras
## 7.1 Definir el modelo
Usaremos un modelo simple de red neuronal con capas densas.

In [ ]:
model = Sequential()
model.add(Dense(128, activation='relu', input_dim=num_movies))
model.add(Dense(256, activation='relu'))
model.add(Dense(num_movies, activation='sigmoid'))  # Usamos sigmoid para salida multi-etiqueta


## 7.2 Compilar el modelo
Usamos la función de pérdida binary_crossentropy y el optimizador adam.

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy', Precision(), Recall()])


## 7.3 Entrenar el modelo
Incluimos EarlyStopping para evitar sobreajuste.

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

model.fit(X_train, Y_train, epochs=20, batch_size=32, validation_split=0.1, callbacks=[early_stopping])


# Paso 8: Evaluar el modelo en el conjunto de prueba

In [ ]:
loss, accuracy, precision, recall = model.evaluate(X_test, Y_test)

print(f"Loss: {loss:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")


# Paso 9: Obtener métricas adicionales
Podemos calcular métricas como el F1-score.

In [ ]:
from sklearn.metrics import f1_score

# Predicciones en el conjunto de prueba
Y_pred_prob = model.predict(X_test)
Y_pred = (Y_pred_prob > 0.5).astype(int)

# Calculamos el F1-score para cada muestra y luego hacemos la media
f1 = f1_score(Y_test, Y_pred, average='samples')

print(f"F1 Score: {f1:.4f}")
